# Importing libraries

In [ ]:
!pip install datasets transformers[torch] sentencepiece
# !pip install torch-xla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.6 MB/s eta 0:00:00


In [ ]:
import json
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import torch
# import torch_xla.core.xla_model as xm

# Defining the model

In [ ]:
# device = xm.xla_device()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


# Preparing data

## Loading the dataset

In [ ]:
#this dataset contains natural language questions and SQL queries
dataset = load_dataset('spider')

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Dataset spider downloaded and prepared to /root/.cache/huggingface/datasets/spider/spider/1.0.0/4e5143d825a3895451569c8b9b55432b91a4bc2d04d390376c950837f4680daa. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download the [Spider dataset zip file](https://drive.google.com/uc?export=download&id=1TqleXec_OykOYFREKKtschzY29dUcVAQ), save it in your Google Drive or Colab disk, and specify the path to it below. If there is space in the path, you must surround the path with double quotes.

In [ ]:
path_to_spider_zip = '"/content/drive/MyDrive/HUST/Natural Language Processing/spider.zip"'

In [ ]:
!unzip {path_to_spider_zip} -d '/content'

Archive:  /content/drive/MyDrive/HUST/Natural Language Processing/spider.zip
   creating: /content/spider/
   creating: /content/spider/database/
   creating: /content/spider/database/customer_deliveries/
  inflating: /content/spider/database/customer_deliveries/schema.sql  
  inflating: /content/spider/database/customer_deliveries/customer_deliveries.sqlite  
   creating: /content/spider/database/allergy_1/
  inflating: /content/spider/database/allergy_1/schema.sql  
  inflating: /content/spider/database/allergy_1/allergy_1.sqlite  
   creating: /content/spider/database/company_office/
  inflating: /content/spider/database/company_office/schema.sql  
  inflating: /content/spider/database/company_office/company_office.sqlite  
   creating: /content/spider/database/device/
  inflating: /content/spider/database/device/schema.sql  
  inflating: /content/spider/database/device/device.sqlite  
   creating: /content/spider/database/phone_1/
  inflating: /content/spider/database/phone_1/schem

In [ ]:
#this dataset contains database schemas
with open('/content/spider/tables.json') as f:
    tables = json.load(f)
schema_dict = {table['db_id']: table for table in tables}

## Preprocessing the dataset

In [ ]:
def preprocess(batch, tokenizer):
    """
    Transforms each training example input into a string of the following format:

    `<question> (" | " <table_name> "(" <column_name> (", " <column_name>)* ")")+`

    and then tokenizes this input representation as well as the ground truth SQL query.

    For example: `List the name, born state and age of the heads of departments ordered by age. | department(department id, name, creation, ranking, budget in billions, num employees) | head(head id, name, born state, age) | management(department id, head id, temporary acting)`

    The format description above is in Python BNF (see https://docs.python.org/3/reference/introduction.html#notation).
    """
    questions = batch['question']
    queries = batch['query']
    db_ids = batch['db_id']
    input_ids = []
    attention_masks = []
    labels = []
    for question, query, db_id in zip(questions, queries, db_ids):
        schema = schema_dict[db_id]
        schema_tokens = []
        for table_id, table_name in enumerate(schema['table_names']):
            schema_tokens.extend([' | ', table_name, '('])
            column_names = [column_name for column_id, column_name in schema['column_names'] if column_id == table_id]
            schema_tokens.append(', '.join(column_names) + ')')
        input_text = ' '.join([question] + schema_tokens)
        input_tokenized = tokenizer(input_text, padding='max_length', truncation=True, max_length=512)
        query_tokenized = tokenizer(query, padding='max_length', truncation=True, max_length=512)
        input_ids.append(input_tokenized['input_ids'])
        attention_masks.append(input_tokenized['attention_mask'])
        labels.append(query_tokenized['input_ids'])
    return {'input_ids': input_ids, 'attention_mask': attention_masks, 'labels': labels}


In [ ]:
tokenized_dataset = dataset.map(lambda batch: preprocess(batch, tokenizer), batched=True, batch_size=32)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

# Training the model

Specify the directory to save model predictions and checkpoints below.

In [ ]:
output_dir_path = '/content/drive/MyDrive/HUST/Natural Language Processing/text2sql/finetuning/output'

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir_path,
    evaluation_strategy='epoch',
    learning_rate=0.001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    weight_decay=0.001,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.137600,0.084323
2,0.026600,0.109886
3,0.018700,0.123427
4,0.014200,0.171995
5,0.011900,0.182669
6,0.010000,0.194796
7,0.008300,0.286855
8,0.006900,0.239056
9,0.005800,0.198768
10,0.005000,0.196212
